노원구 환경기상 데이터 전처리
-------------------------------------------------------------------------------
> 1. 측정소마다 저장된 데이터를 하나의 파일로 통합
> 2. 법정동이름을 현업의 편의상 지정된 코드로 바꿔줌
> 3. 결측치, 이상치 제거

In [33]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import glob
import os
directory = "./Desktop/dataset/bigcon/환경기상데이터/"

In [34]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# 노트북 안에 그래프를 그리기 위해
%matplotlib inline

# 그래프에서 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일을 사용
plt.style.use('ggplot')

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

데이터 불러오기
-------------------------------------------------------------------------------
>  측정소마다 저장된 데이터를 하나의 파일로 통합하기 위하여 데이터를 불러와 concat하여줍니다.


In [35]:
read_file_nowon_lst = os.listdir('./Desktop/dataset/bigcon/환경기상데이터/노원구/')
read_df_nowon_lst = []
print ("read_file_nowon_lst: {}".format(read_file_nowon_lst))

for f in read_file_nowon_lst:
    if f == '.ipynb_checkpoints':
        pass
    else :
        df_temp = pd.read_csv(directory + '노원구/' + f)
        read_df_nowon_lst.append(df_temp)


read_file_nowon_lst: ['V10O1610629.csv', 'V10O1610616.csv', 'V10O1611652.csv', 'V10O1610376.csv', 'V10O1611097.csv', 'V10O1610610.csv', 'V10O1610200.csv', 'V10O1610312.csv', 'V10O1611150.csv', 'V10O1610102.csv', 'V10O1610643.csv', 'V10O1610642.csv', 'V10O1610293.csv', 'V10O1611229.csv', 'V10O1612126.csv', 'V10O1610297.csv', '.ipynb_checkpoints', 'V10O1610351.csv', 'V10O1611104.csv', 'V10O1610356.csv', 'V10O1611100.csv', 'V10O1611102.csv', 'V10O1610630.csv']


In [36]:
nowon = pd.concat(read_df_nowon_lst)

In [37]:
nowon.head()

,tm,serial,flag,pm10,co2,vocs,noise,temp,humi,pm25,Unnamed: 10
0,201804010000,V10O1610629,1,32,-9999,-9999,53,15.2,52,16.0,NaN
1,201804010001,V10O1610629,1,25,-9999,-9999,57,15.1,52,12.0,NaN
2,201804010002,V10O1610629,1,30,-9999,-9999,50,15.1,53,15.0,NaN
3,201804010003,V10O1610629,1,32,-9999,-9999,58,15.1,53,16.0,NaN
4,201804010004,V10O1610629,1,33,-9999,-9999,53,15.2,53,16.0,NaN


날짜 칼럼 이름 변경
-------------------------------------------------------------------------------
>  데이터의 통합을 위하여 tm으로 된 칼럼을 Y/M/D/h별로 나누어 줍니다.

In [38]:
nowon["Year"] = nowon["tm"]
nowon["Month"] = nowon["tm"]
nowon["Days"] = nowon["tm"]
nowon["hours"] = nowon["tm"]
days = [int(str(mmdd)[6:8]) for mmdd in nowon["tm"]]
month = [int(str(mmdd)[4:6]) for mmdd in nowon["tm"]]
year = [int(str(mmdd)[0:4]) for mmdd in nowon["tm"]]
hours = [int(str(mmdd)[8:10]) for mmdd in nowon["tm"]]

nowon["Days"] = days
nowon["Year"] = year
nowon["Month"] = month
nowon["hours"] = hours
nowon.head()

,tm,serial,flag,pm10,co2,vocs,noise,temp,humi,pm25,Unnamed: 10,Year,Month,Days,hours
0,201804010000,V10O1610629,1,32,-9999,-9999,53,15.2,52,16.0,NaN,2018,4,1,0
1,201804010001,V10O1610629,1,25,-9999,-9999,57,15.1,52,12.0,NaN,2018,4,1,0
2,201804010002,V10O1610629,1,30,-9999,-9999,50,15.1,53,15.0,NaN,2018,4,1,0
3,201804010003,V10O1610629,1,32,-9999,-9999,58,15.1,53,16.0,NaN,2018,4,1,0
4,201804010004,V10O1610629,1,33,-9999,-9999,53,15.2,53,16.0,NaN,2018,4,1,0


측정소 칼럼을 법정동 이름으로 변경
-------------------------------------------------------------------------------
>  데이터의 통합을 위하여 serial에 기록된 측정소 칼럼을 법정동이름으로 바꾸어 줍니다.

In [39]:
nowon_idx = ['공릉1동','공릉1동','공릉2동','공릉2동','상계10동','상계1동','상계2동','상계2동','상계2동','상계2동','상계3,4동','상계5동','상계5동','상계6,7동','상계6,7동','상계6,7동','월계1동','월계3동','중계2,3동','중계본동','하계1동','하계1동']
nowon_station = ['V10O1610629','V10O1610630','V10O1610642','V10O1611652','V10O1610293','V10O1610356','V10O1610616','V10O1610200','V10O1610312','V10O1611100','V10O1611150','V10O1610297','V10O1611102','V10O1610376','V10O1610351','V10O1611104','V10O1611229','V10O1610643','V10O1611097','V10O1612126','V10O1610610','V10O1610102']
nowon_station_idx = dict(zip(nowon_station, nowon_idx))

for key in nowon_station_idx.keys():
    nowon['serial'] = np.where(nowon['serial'] == key, nowon_station_idx[key], nowon['serial'])

nowon.head()


,tm,serial,flag,pm10,co2,vocs,noise,temp,humi,pm25,Unnamed: 10,Year,Month,Days,hours
0,201804010000,공릉1동,1,32,-9999,-9999,53,15.2,52,16.0,NaN,2018,4,1,0
1,201804010001,공릉1동,1,25,-9999,-9999,57,15.1,52,12.0,NaN,2018,4,1,0
2,201804010002,공릉1동,1,30,-9999,-9999,50,15.1,53,15.0,NaN,2018,4,1,0
3,201804010003,공릉1동,1,32,-9999,-9999,58,15.1,53,16.0,NaN,2018,4,1,0
4,201804010004,공릉1동,1,33,-9999,-9999,53,15.2,53,16.0,NaN,2018,4,1,0


nowon.isnull().sum()

In [40]:
nowon_copy = nowon

In [41]:
nowon_copy = nowon[['Year','Month','Days','hours','serial','pm10','pm25']]

In [42]:
nowon_copy.head()

,Year,Month,Days,hours,serial,pm10,pm25
0,2018,4,1,0,공릉1동,32,16.0
1,2018,4,1,0,공릉1동,25,12.0
2,2018,4,1,0,공릉1동,30,15.0
3,2018,4,1,0,공릉1동,32,16.0
4,2018,4,1,0,공릉1동,33,16.0


결측치와 이상치 제거
-------------------------------------------------------------------------------
>  결측치와 이상치를 확인하고 제거해줍니다.

In [43]:
#결측치 확인하기
nowon_copy.isnull().sum()

Year           0
Month          0
Days           0
hours          0
serial         0
pm10           0
pm25      151925
dtype: int64

In [44]:
#이상치 확인하기
nowon_copy[nowon_copy['pm25']< 0].sum()


Year                                               37544512
Month                                                139830
Days                                                 296661
hours                                                197885
serial    공릉1동공릉1동공릉1동공릉1동공릉1동공릉1동공릉1동공릉1동공릉1동공릉1동상계2동상계...
pm10                                              -18585396
pm25                                           -1.85854e+07
dtype: object

In [45]:
# 이상치 결측치로 처리하기

nowon_copy.loc[nowon_copy['pm25'] < 0, 'pm25'] = None

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [46]:
nowon_copy.isnull().sum()

Year           0
Month          0
Days           0
hours          0
serial         0
pm10           0
pm25      170529
dtype: int64

In [47]:
nowon_copy.loc[nowon_copy['pm10'] < 0, 'pm10'] = np.nan

In [48]:
nowon_copy.isnull().sum()

Year           0
Month          0
Days           0
hours          0
serial         0
pm10      170529
pm25      170529
dtype: int64

In [49]:
nowon_copy2 = nowon_copy.copy()

In [50]:
nowon_copy2 = nowon_copy2.dropna()

In [51]:
nowon_copy2.isnull().sum()

Year      0
Month     0
Days      0
hours     0
serial    0
pm10      0
pm25      0
dtype: int64

In [52]:
nowon_copy2.isnull().sum()

Year      0
Month     0
Days      0
hours     0
serial    0
pm10      0
pm25      0
dtype: int64

In [53]:
nowon_copy3 = nowon_copy2.groupby(['serial','Year', 'Month','Days','hours'])['pm25','pm10'].mean().astype(int)

In [54]:
nowon_copy3

pm25  pm10
serial Year Month Days hours            
공릉1동   2018 4     1    0        16    33
                       1        17    38
                       2        20    42
                       3        16    31
                       4        16    30
                       5        19    36
                       6        23    43
                       7        20    39
                       8        30    58
                       9        28    61
                       10       24    61
                       11       24    63
                       12       22    57
                       13       17    45
                       14       18    43
                       15       22    45
                       16       33    64
                       17       36    66
                       18       37    66
                       19       36    63
                       20       35    66
                       21       36    72
                       22       35    74
                       23       32    69
                  2    0        31    68
                       1        28    66
                       2        25    63
                       3        24    63
                       4        25    63
                       5        24    59
...                            ...   ...
하계1동   2019 3     30   18       11    24
                       19       18    28
                       20       24    30
                       21       27    33
                       22       27    34
                       23       21    26
                  31   0        15    19
                       1        18    21
                       2        20    22
                       3        23    26
                       4        21    24
                       5        24    26
                       6        31    33
                       7        30    36
                       8        31    38
                       9        32    42
                       10       34    49
                       11       35    54
                       12       33    55
                       13       29    54
                       14       28    54
                       15       25    45
                       16       18    30
                       17       16    26
                       18       11    20
                       19        9    19
                       20        9    17
                       21       18    26
                       22       19    27
                       23       18    28

[103621 rows x 2 columns]

In [55]:
nowon_copy3.to_csv('pre_nowon.csv')

In [56]:
pre_nowon = pd.read_csv('pre_nowon.csv')
pre_nowon

,serial,Year,Month,Days,hours,pm25,pm10
0,공릉1동,2018,4,1,0,16,33
1,공릉1동,2018,4,1,1,17,38
2,공릉1동,2018,4,1,2,20,42
3,공릉1동,2018,4,1,3,16,31
4,공릉1동,2018,4,1,4,16,30
5,공릉1동,2018,4,1,5,19,36
6,공릉1동,2018,4,1,6,23,43
7,공릉1동,2018,4,1,7,20,39
8,공릉1동,2018,4,1,8,30,58
9,공릉1동,2018,4,1,9,28,61


법정동 칼럼을 지역코드로 변경
-------------------------------------------------------------------------------
>  데이터의 통합을 위하여 serial에 기록된 법정동 칼럼을 지역코드로 바꾸어 줍니다.

In [57]:
nowon_code = {11350560: "월계1동",
11350570: "월계2동",
11350580: "월계3동",
11350595: "공릉1동",
11350600: "공릉2동",
11350611: "하계1동",
11350612: "하계2동",
11350619: "중계본동",
11350621: "중계1동",
11350624: "중계4동",
11350625: "중계2.3동",
11350630: "상계1동",
11350640: "상계2동",
11350665: "상계3.4동,",
11350670: "상계5동",
11350695: "상계6.7동,",
11350700: "상계8동",
11350710: "상계9동",
11350720: "상계10동",}

for key in nowon_code.keys():
    pre_nowon['serial'] = np.where(pre_nowon['serial'] == nowon_code[key], key, pre_nowon['serial'])


In [58]:
pre_nowon

,serial,Year,Month,Days,hours,pm25,pm10
0,11350595,2018,4,1,0,16,33
1,11350595,2018,4,1,1,17,38
2,11350595,2018,4,1,2,20,42
3,11350595,2018,4,1,3,16,31
4,11350595,2018,4,1,4,16,30
5,11350595,2018,4,1,5,19,36
6,11350595,2018,4,1,6,23,43
7,11350595,2018,4,1,7,20,39
8,11350595,2018,4,1,8,30,58
9,11350595,2018,4,1,9,28,61


In [59]:
pre_nowon.to_csv('pre_nowon.csv', index = False)
pre_nowon = pd.read_csv('pre_nowon.csv')
pre_nowon

,serial,Year,Month,Days,hours,pm25,pm10
0,11350595,2018,4,1,0,16,33
1,11350595,2018,4,1,1,17,38
2,11350595,2018,4,1,2,20,42
3,11350595,2018,4,1,3,16,31
4,11350595,2018,4,1,4,16,30
5,11350595,2018,4,1,5,19,36
6,11350595,2018,4,1,6,23,43
7,11350595,2018,4,1,7,20,39
8,11350595,2018,4,1,8,30,58
9,11350595,2018,4,1,9,28,61
